In [ ]:
import json
import os

ROOT_DIR = os.path.abspath(__vsc_ipynb_file__)

data = """
    {
    "CRC": "12345678",
    "VID": "TRACT01",
    "command": 80100,
    "data": {
        "autodrivingstatus": "1",
        "brakestatus": 0,
        "drivingDistance": "",
        "emergencyLight": "",
        "gearstatus": 0,
        "mdpsAngle": "",
        "turnSignal": "",
        "wheelSpeed": "",
        "yawRate": ""
    },
    "timestamp": "2024-07-26 02:01:08.741",
    "type": "VEHICLE_INFO"
    }
    """
    
output_file = os.path.join('log',"tractor.log")
data1 = json.loads(data)
# print(data1)

with open(output_file,'a',encoding='utf-8') as of:
    json.dump(data1,of)
    of.write("\n")

pars_data = data1['data']
print(pars_data)
#print(data1)

#data2 = json.dumps(data1)




In [ ]:
import json
import os

ROOT_DIR = os.path.abspath(__vsc_ipynb_file__)
file_name = 'SweeperInfoJM.log'
output_file = os.path.join('log',"tractor1.log")


with open(file_name,'r',encoding='utf-8') as f:
    data = f.readlines()

for i in data:
    data = json.loads(i)
    
    with open(output_file,'a',encoding='utf-8') as of:
        json.dump(data,of,ensure_ascii=False)
        of.write("\n")
        
print(data1)

In [ ]:
import os
import json
import sys
import socket
import json
import time
from datetime import datetime
import pandas as pd
import paho.mqtt.client as mqtt
import psycopg2

def receive_data():
    host = '0.0.0.0'
    port = 5000
    buffer = ""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((host, port))
        print(f"Connected to {host}:{port}")
        while True:
            data = s.recv(4096)
            if not data:
                break
            buffer += data.decode('utf-8')
            while True:
                try:
                    obj, idx = json.JSONDecoder().raw_decode(buffer)
                    buffer = buffer[idx:].lstrip()
                    yield obj
                except json.JSONDecodeError:
                    break

# 평균값 계산
def calculate_average(data_points):
    if not data_points:
        return {}
    vehicle_data = []
    location_data = []
    for batch in data_points:
        for item in batch:
            if item['type'] == 'VEHICLE_INFO':
                vehicle_data.append(item['data'])
            if item['type'] == 'LOCATION':
                location_data.append(item['data'])
                
    vehicle_df = pd.DataFrame(vehicle_data)
    location_df = pd.DataFrame(location_data)
    location_df.columns = ['accel', 'autoDrivingStatus', 'brake', 'brakeStatus', 'drivingDistance',
       'emergencyLight', 'mdpsAngle', 'transmission', 'turnSignal',
       'wheelSpeed', 'yawRate', 'l_latitude', 'l_longitude']
    
    
    comm_df = pd.concat([vehicle_df,location_df],axis=1)
    comm_df = comm_df.apply(pd.to_numeric, errors='coerce')
    comm_df = comm_df.mean()
    comm_df['v_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    comm_df['l_timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    comm_df['vehicle_id'] = 'STSW000002'
    comm_df['vehicle_type'] = 'S'

    comm_df = comm_df.to_json()
    avg_data = [comm_df]
    print(avg_data)
    
    return avg_data    

#mqtt 발행
def send_to_cloud(avg_data):
    broker_address = "223.130.131.234"
    port = 31883
    topic = "special_vehicle/data"

    client = mqtt.Client("EdgeNode")
    client.connect(broker_address, port)
    client.publish(topic, json.dumps(avg_data))
    client.disconnect()
                
def process_data():
    data_points = []
    start_time = time.time()

    for data in receive_data():
        data_points.append(data)
        #save_to_db(data)
        #print(data)
        if time.time() - start_time >= 1:
            avg_data = calculate_average(data_points)
            send_to_cloud(avg_data)
            data_points = []            
            start_time = time.time()

if __name__ == "__main__":
    process_data()

In [10]:
import socketserver
import logging
import datetime
import configparser
from pathlib import Path
import os
from logging.handlers import TimedRotatingFileHandler
import json
import sys
import socket
import time
from datetime import datetime
import pandas as pd
import paho.mqtt.client as mqtt
import psycopg2

client_socket = []

recv_encoding = 'CP949'


class MyTCPHandler(socketserver.BaseRequestHandler):
    def __init__(self, request,client_address,server):
        socketserver.BaseRequestHandler.__init__(self,request,client_address,server)
        
        return
    
    def setup(self):
        return socketserver.BaseRequestHandler.setup(self)
    
    
    def handle(self):
        conn = self.request
        addr = self.client_address[0]
        recv_len = 4
        
        client_socket.append((conn,addr))
        buf = ""
        
        

if __name__ == "__main__":
    host = "0.0.0.0"
    port = 5000
    
    try:
        server = (socket.gethostbyname(host), port), MyTCPHandler
    except ValueError:
        print("ValueError Closing")        
    except :
        print("Bind Failed. Closing")

In [47]:
import json
import datetime
#from constant import Constant


with open("command_tbl.txt", 'r', encoding='utf-8') as file:
    command_tbl = json.load(file)
data = """
    {
    "CRC": "12345678",
    "VID": "TRACT01",
    "command": 80000,
    "data": {
        "autodrivingstatus": "1",
        "brakestatus": 0,
        "drivingDistance": "",
        "emergencyLight": "",
        "gearstatus": 0,
        "mdpsAngle": "",
        "turnSignal": "",
        "wheelSpeed": "",
        "yawRate": ""
    },
    "timestamp": "2024-07-26 02:01:08.741",
    "type": "VEHICLE_INFO"
    }\\n
    """

for i in range(1):
    buf = ""
    buf += data

    index = buf.find("\\n")
    if index == -1:
        print("index -1")
        continue
    else:
        now = datetime.datetime.now()
        
        data = buf[0:index +2]
        data = data.replace("\\n", '')
        buf = data.replace(data, '')
        
        send_message = ""
        json_message = dict()
        res_repack = dict()
        
    try:
        res = json.loads(str(data))
        
        edge_cmd = str(res["command"])
        
        try:
            cmd_mapping = command_tbl[edge_cmd]
        except KeyError as e:
            print("존재하지 않는 command")
            continue
        
        header_repack = dict()
        
        header_repack["cmd"] = str(cmd_mapping["cmd"])
        header_repack["actn"] = str(cmd_mapping["actn"])
        header_repack["dtlActn"] = str(cmd_mapping["dtlActn"])
        header_repack["strtpnt"] = "E"
        header_repack["dstn"] = "N"
        header_repack["userId"] = ""
        header_repack["edgeId"] = (res["VID"])
        #header_repack["edgeTy"] = (res["edgeTy"])
        header_repack["timestamp"] = (res["timestamp"])
        header_repack["command"] = edge_cmd
                
        res_repack["header"] = header_repack
        res_repack["data"] = res["data"]
        
        _cmd = str(res_repack["header"]["cmd"])
        _actn = str(res_repack["header"]["actn"])
        _dtlActn = str(res_repack["header"]["dtlActn"])
        _strtpnt = str(res_repack["header"]["strtpnt"])
        _dstn = str(res_repack["header"]["dstn"])
        _userId = str(res_repack["header"]["userId"])
        _edgeId = str(res_repack["header"]["edgeId"])
        #_edgeTy = str(res_repack["header"]["edgeTy"])
        _timestamp = str(res_repack["header"]["timestamp"])
        
        
        
        
        start_time = datetime.datetime.strptime(_timestamp,'%Y-%m-%d %H:%M:%S.%f')
        delay_time = (now - start_time) * 1000
        
        _cmd_req = _cmd
        _strtpnt_res = _dstn
        _dstn_res = "H"
        
        resheader = res_repack["header"]
        resdata = dict()
        #send_direction = Constant.NONE
        
        if _cmd == "req":
            if _actn == "status":
            if _actn == "alarm":
            if _actn == "globalpath":
            if _actn == "trc":                
            if _actn == "cls":
                resheader["cmd"] = _cmd_req
                resheader["strtpnt"] = _strtpnt_res
                resheader["dstn"] = _dstn_res
                
                json_message["header"] = resheader
                data_message = str(json.dumps(res["data"], ensure_ascii=False))
                print(_timestamp,_edgeId,data_message)
        
        if _cmd == "rep":
            if _actn == "init":
            if _actn == "status":                
            if _actn == "alarm":                
            if _actn == "globalpath":                
            if _actn == "trc":                
            if _actn == "cls":                
                
                
    except json.decoder.JSONDecodeError as e:
        print(e)
    except Exception as e:
        print(e)

2024-07-26 02:01:08.741 TRACT01 {"autodrivingstatus": "1", "brakestatus": 0, "drivingDistance": "", "emergencyLight": "", "gearstatus": 0, "mdpsAngle": "", "turnSignal": "", "wheelSpeed": "", "yawRate": ""}
